In [37]:
'''
This returns a list of the unique elements within a longer list. 
'''

def unique(input_list): 
    
    # Null list 
    unique_list = []
    
    # Traverse 
    for element in input_list: 
        if element in unique_list: 
            pass
        else: 
            unique_list.append(element)
    
    return unique_list

In [ ]:
'''
Exports a feature class, clips it by the polygon feature class, performs a spatial join, and dissolves by polygon name. 
In a situation where there are contiguous polygons (as opposed to state parks, where they are separate)
'''

def aggregate_data_by_park(input_path, sql, gdb, fc_name, state_parks): 
    
    # Print 
    print("---------------------------")
    print(fc_name)
    print("---------------------------")

    # Export 
    feature_class = os.path.join(gdb, fc_name)
    if arcpy.Exists(feature_class) == False: 
        arcpy.conversion.ExportFeatures(input_path, feature_class, sql)
        print(f"Exported")
    
    # Clip 
    clipped_fc    = os.path.join(gdb, "Clipped_" + fc_name)
    if arcpy.Exists(clipped_fc) == False: 
        arcpy.analysis.Clip(feature_class, state_parks, clipped_fc)
        print(f"Clipped")
    
    # Spatial join 
    joined_fc     = os.path.join(gdb, "Joined_" + fc_name)
    if arcpy.Exists(joined_fc) == False: 
        arcpy.analysis.SpatialJoin(clipped_fc, state_parks, joined_fc, 'JOIN_ONE_TO_ONE', match_option = 'HAVE_THEIR_CENTER_IN')
        print(f"Spatial Joined")
    
    # Dissolve by park name 
    by_park_fc    = os.path.join(gdb, fc_name + "_By_Park")
    if arcpy.Exists(by_park_fc) == False: 
        arcpy.management.Dissolve(joined_fc, by_park_fc, 'AREA_NAME')
        print(f"Dissolved")
        
    return joined_fc, by_park_fc

In [ ]:
'''
Creates a dictionary of park values and list of attributes. 
'''

def create_list_by_feature(joined_features, polygon_name_field, list_feature_field): 
    
    # Initialize dictionary
    list_by_feature = {}

    # Iterate by row and initialize dictionary key (park)
    with arcpy.da.SearchCursor(old_growth, ['AREA_NAME', 'MAIN_SP_DESC']) as cursor: 
        for row in cursor: 

            # Identify attributes 
            park                    = row[0]
            list_by_feature[park] = []

    # Iterate by row and initialize dictionary key (park)
    with arcpy.da.SearchCursor(old_growth, ['AREA_NAME', 'MAIN_SP_DESC']) as cursor: 
        for row in cursor: 

            # Append species to value list of park key
            try: 
                list_by_feature[row[0]].append("(" + row[1] + ")")
            except: 
                pass

    # Reduce values to unique values 
    for park, species_list in list_by_feature.items(): 
        list_by_feature[park] = unique(list_by_feature[park])
        
    # Create dataframe 
    dic = {} 
    dic['AREA_NAME']    = list(list_by_feature.keys())
    dic[list_feature_field] = list(list_by_feature.values())
        
    return dic

In [ ]:
def group_by_park_dict_to_df(group_by_park_dict, group_name): 
    
    # Initialize dataframe 
    dataframe_dict = {} 
    dataframe_dict['Park']     = []
    dataframe_dict[group_name] = []
    
    # Iterate through features 
    for park, group in group_by_park_dict.items(): 
        dataframe_dict['Park'].append(park)
        dataframe_dict[group_name].append(group)

    df = pd.DataFrame.from_dict(dataframe_dict)
    
    return df 

In [ ]:
'''
Imports 
'''
import os 
import pandas as pd 
import arcgis

In [8]:
'''
Map 
'''
# Initialize 
aprx = arcpy.mp.ArcGISProject('CURRENT')
mp   = aprx.listMaps('State Parks')[0]

# Clear map 
for lyr in mp.listLayers(): 
    mp.removeLayer(lyr)
    
# Add baselayer
mp.addDataFromPath(r"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Census_States/FeatureServer")
mp.addDataFromPath(r"C:\DALE\Personal\Minnesota\MyProject\Import Data\bdry_dnr_lrs_prk.gdb\dnr_stat_plan_areas_prk")

In [ ]:
'''
Geodatabase 
'''
# Initialize 
project_folder = r"C:\DALE\Personal\Minnesota\MyProject"
gdb_name       = r"State_Parks.gdb"

# Create geodatabase 
gdb_path = os.path.join(project_folder, gdb_name)

if arcpy.Exists(gdb_path) == False: 
    gdb = arcpy.management.CreateFileGDB(project_folder, gdb_name)
    print(f'Geodatabase created: {gdb_path}')
    
else: 
    gdb = os.path.join(project_folder, gdb_name)
    print(f'Geodatabase exists: {gdb}')
    
# Clear geodatabase
clear_gdb           = False
arcpy.env.workspace = gdb_path

if clear_gdb == True: 
    for feature_class in arcpy.ListFeatureClasses(): 
        fc_path = os.path.join(gdb, feature_class)
        arcpy.management.Delete(fc_path)

In [ ]:
##################################################################################################
##################################################################################################
##################################################################################################

In [ ]:
'''
State Park Field Dictionary
'''
state_park_field_dictionary = {} 

state_park_field_dictionary['hike_miles']                = {} 
state_park_field_dictionary['hike_miles']['alias']       = 'Hiking Miles'
state_park_field_dictionary['hike_miles']['field_type']  = 'FLOAT'

state_park_field_dictionary['trout_mi']                  = {}
state_park_field_dictionary['trout_mi']['alias']         = 'Trout Stream Miles'
state_park_field_dictionary['trout_mi']['field_type']    = 'FLOAT'

state_park_field_dictionary['bike_mi']                   = {}
state_park_field_dictionary['bike_mi']['alias']          = 'Biking Miles'
state_park_field_dictionary['bike_mi']['field_type']     = 'FLOAT'

state_park_field_dictionary['biome']                     = {}
state_park_field_dictionary['biome']['alias']            = 'Biome'
state_park_field_dictionary['biome']['field_type']       = 'TEXT'

state_park_field_dictionary['drv_tim']                   = {}
state_park_field_dictionary['drv_tim']['alias']          = 'Driving Distance'
state_park_field_dictionary['drv_tim']['field_type']     = 'FLOAT'

In [260]:
'''
State parks 
'''

# Export to new feature class 
state_parks_path    = r"C:\DALE\Personal\Minnesota\MyProject\Import Data\bdry_dnr_lrs_prk.gdb\dnr_stat_plan_areas_prk"
state_parks_sql     = "1=1"
state_parks         = os.path.join(gdb, 'State_Parks')
arcpy.conversion.ExportFeatures(state_parks_path, state_parks, state_parks_sql) 

# Get rid of apostrophes in names 
with arcpy.da.UpdateCursor(state_parks, ['AREA_NAME']) as cursor: 
    for row in cursor:
        if "'" in row[0]: 
            row[0] = row[0].replace("'", "")
            cursor.updateRow(row)
            
# Get rid of unnecessary columns 
arcpy.management.DeleteField(state_parks, drop_field = ['AREA_NAME', 'AREA_TYPE'], method = 'KEEP_FIELDS')
mp.addDataFromPath(state_parks)

In [ ]:
###################################################################################################################################
###############################                                                       #############################################
###############################         Above this initializes the project            #############################################
###############################                                                       #############################################
###################################################################################################################################

In [ ]:
'''
Input Groups 
'''

# Initialize input dictionary 
park_attribute_dictionary = {} 

park_attribute_dictionary["Old_Growth_Forest"] = {}
park_attribute_dictionary["Old_Growth_Forest"]["SQL"] = "1=1"
park_attribute_dictionary["Old_Growth_Forest"]["PATH"] = r"C:\DALE\Personal\Minnesota\MyProject\Import Data\biota_dnr_forest_inv_old_growth.gdb\dnr_forest_inventory_old_growth"
park_attribute_dictionary["Old_Growth_Forest"]["GEOMETRY_ATTRIBUTE"] = 'AREA'
park_attribute_dictionary["Old_Growth_Forest"]["GROUP_FIELD"] = 'MAIN_SP_DESC'

park_attribute_dictionary["Hiking_Trails"] = {}
park_attribute_dictionary["Hiking_Trails"]["SQL"] = "use_hike = 1 And use_mntbik = 0 And surfacetyp = 'Natural soil surface type - no improvements made' And use_atvcl1 = 0 And use_atvcl2 = 0"
park_attribute_dictionary["Hiking_Trails"]["PATH"] = r"C:\DALE\Personal\Minnesota\MyProject\Import Data\trans_state_park_trails_roads.gdb\state_park_trails_roads"
park_attribute_dictionary["Hiking_Trails"]["GEOMETRY_ATTRIBUTE"] = 'LENGTH'
park_attribute_dictionary["Hiking_Trails"]["GROUP_FIELD"] = 'NONE'

park_attribute_dictionary["Trout_Streams"] = {}
park_attribute_dictionary["Trout_Streams"]["SQL"] = "1=1"
park_attribute_dictionary["Trout_Streams"]["PATH"] = r"C:\DALE\Personal\Minnesota\MyProject\Import Data\env_trout_stream_designations.gdb\trout_streams_minnesota"
park_attribute_dictionary["Trout_Streams"]["GEOMETRY_ATTRIBUTE"] = 'LENGTH'
park_attribute_dictionary["Trout_Streams"]["GROUP_FIELD"] = 'NONE'

park_attribute_dictionary['State_Bike_Trails'] = {} 
park_attribute_dictionary['State_Bike_Trails']['SQL']                 = 'use_bike = 1'
park_attribute_dictionary['State_Bike_Trails']['PATH']                = r"C:\DALE\Personal\Minnesota\MyProject\Import Data\trans_state_trails_minnesota.gdb\state_trails_minnesota"
park_attribute_dictionary['State_Bike_Trails']['GEOMETRY_ATTRIBUTE']  = 'LENGTH'
park_attribute_dictionary['State_Bike_Trails']['GROUP_FIELD']         = 'NONE'

park_attribute_dictionary['Native_Plant_Communities'] = {} 
park_attribute_dictionary['Native_Plant_Communities']['SQL']                 = '1 = 1'
park_attribute_dictionary['Native_Plant_Communities']['PATH']                = r"C:\DALE\Personal\Minnesota\MyProject\Import Data\biota_dnr_native_plant_comm.gdb\dnr_native_plant_communities"
park_attribute_dictionary['Native_Plant_Communities']['GEOMETRY_ATTRIBUTE']  = 'AREA'
park_attribute_dictionary['Native_Plant_Communities']['GROUP_FIELD']         = 'NPC_description'

In [ ]:
###################################################################################################################################
###############################                                                       #############################################
###############################         This creates the dictionary of inputs         #############################################
###############################                                                       #############################################
###################################################################################################################################

In [ ]:
'''
Create individual feature classes 
'''

# Iterate through dictionary 
area_features = [] 
mile_features = []
existing_columns = []

# Join column 
join_column = 'AREA_NAME'

# Create spatial dataframe 
state_parks_sdf = pd.DataFrame.spatial.from_featureclass(state_parks)
go              = True 

master_sdf      = pd.DataFrame() 
master_group_df = pd.DataFrame() 

# Iterate through the input dictionary
for fc_name, attributes in park_attribute_dictionary.items(): 
    
    # Identify attributes 
    sql                = attributes['SQL']
    input_path         = attributes['PATH']
    geometry_attribute = attributes['GEOMETRY_ATTRIBUTE']
    group_field        = attributes['GROUP_FIELD']
    
    # Create outputs 
    joined_features, by_park_features   = aggregate_data_by_park(input_path, sql, gdb, fc_name, state_parks)
    
    # Create group list field 
    if group_field != 'None':
        group_field_merge = True 
        dic = create_list_by_feature(joined_features, 'AREA_NAME', group_field)
        group_df = pd.DataFrame.from_dict(dic)
        
    else: 
        group_field_merge = False 
    
    # Calculate geometry 
    if geometry_attribute == 'AREA': 
        arcpy.management.CalculateGeometryAttributes(by_park_features, [[fc_name + "_Sq_Miles", 'AREA']], area_unit = 'SQUARE_MILES_US')
        
    if geometry_attribute == 'LENGTH': 
        arcpy.management.CalculateGeometryAttributes(by_park_features, [[fc_name + "_Miles", 'LENGTH']], length_unit = 'MILES_US')
    
    # Create spatial dataframe 
    sdf        = pd.DataFrame.spatial.from_featureclass(by_park_features)
    
    # Drop columns
    for column in sdf.columns:
        if column in state_parks_sdf.columns and column != join_column:
            sdf = sdf.drop(columns = column)
    
    # Merge 
    if master_sdf.empty == True: 
        master_sdf = pd.merge(state_parks_sdf, sdf, on = join_column, how = 'outer')
        go = False 
        existing_columns = list(master_sdf.columns)
        
    else: 
        master_sdf = pd.merge(master_sdf, sdf, on = join_column, how = 'outer')
        
    # Merge list dataframes
    if group_field == 'NONE': 
        pass 
    
    else: 
        if master_group_df.empty == True and group_field_merge == True: 
            master_group_df = pd.merge(state_parks_sdf, group_df, on = join_column, how = 'outer')
        elif master_group_df.empty == False and group_field_merge == True: 
            master_group_df = pd.merge(master_group_df, group_df, on = join_column, how = 'outer')
        else: 
            pass

# Drop columns
for column in master_group_df.columns:
    if column in master_sdf.columns and column != join_column:
        master_group_df = master_group_df.drop(columns = column)     
        
# Merge main state park dataframe and group dataframe 
master_sdf = pd.merge(master_sdf, master_group_df, on = join_column, how = 'outer')
master_sdf

# Export to feature class 
attributed_state_parks = os.path.join(gdb, 'Attributed_State_Parks')
master_sdf.spatial.to_featureclass(attributed_state_parks)

In [ ]:
###################################################################################################################################
###############################                                                       #############################################
###############################       This creates the output feature classes         #############################################
###############################                                                       #############################################
###################################################################################################################################

In [258]:
'''
Symbology of outputs 
This will not use the clipped features, but the regular ones 
'''

for a, b in park_attribute_dictionary.items(): 
    path = b['PATH']
    mp.addDataFromPath(path)
    

In [296]:
graphic_elements = []
brick_brown      = [150, 40, 27, 100]
trans_brown      = [150, 40, 27, 20]
lyr_name         = 'State_Parks'

def graphic_element(graphic_elements, border_color, fill_color): 

    # Reference graphic_elements and its CIM definition 
    aprx           = arcpy.mp.ArcGISProject("CURRENT")
    lyr            = mp.listLayers(lyr_name)[0]
    lyr_definition = lyr.getDefinition('V3')
    lyr_symb_lyrs  = lyr_definition.renderer.symbol.symbol.symbolLayers

    # Create stroke color 
    border_color             = arcpy.cim.CreateCIMObjectFromClassName('CIMRGBColor', 'V3')
    border_color.values      = brick_brown

    fill_color               = arcpy.cim.CreateCIMObjectFromClassName('CIMRGBColor', 'V3')
    fill_color.value         = trans_brown

    # Create a symbol layer 
    border_solidstroke       = arcpy.cim.CreateCIMObjectFromClassName('CIMSolidStroke', 'V3')
    border_solidstroke.color = border_color
    border_solidstroke.width = 5

    background_solidfill     = arcpy.cim.CreateCIMObjectFromClassName('CIMSolidStroke', 'V3')
    background_solidfill.color = fill_color

    # Create a polygon symbol 
    polygon_symbol = arcpy.cim.CreateCIMObjectFromClassName('CIMPolygonSymbol', 'V3')
    polygon_symbol.symbolLayers = [background_solidfill, border_solidstroke]

    # Create a symbol reference 
    symbol_reference = arcpy.cim.CreateCIMObjectFromClassName('CIMSymbolReference', 'V3')
    symbol_reference.symbol = polygon_symbol 

    # CIMGraphicElement
    graphic_element = arcpy.cim.CreateCIMObjectFromClassName('CIMGraphicElement', 'V3')

    # Add to graphic_elements elements 
    lyr_symb_lyrs.append(graphic_element)
    lyr.setDefinition(lyr_definition)
    
    return graphic_elements 

graphic_elements = graphic_element(graphic_elements, border_color, fill_color)

In [328]:
help(arcpy.cim.CreateCIMObjectFromClassName())

TypeError: CreateCIMObjectFromClassName() missing 2 required positional arguments: 'cim_class_name' and 'cim_version'

In [23]:
'''
Define colors 
'''
black            = [0, 0, 0, 100]
white            = [255, 255, 255, 100]
blue             = [0, 0, 255,100]
red              = [255, 0, 0, 100]
green            = [0, 255, 0, 100]
brick_brown      = [150, 40, 27, 100]
trans_brown      = [150, 40, 27, 20]

In [ ]:
'''
Define polygon symbol 
'''
# Create stroke color 
border_color             = arcpy.cim.CreateCIMObjectFromClassName('CIMRGBColor', 'V3')
border_color.values      = brick_brown

fill_color               = arcpy.cim.CreateCIMObjectFromClassName('CIMRGBColor', 'V3')
fill_color.value         = green

# Create a symbol layer 
border_solidstroke       = arcpy.cim.CreateCIMObjectFromClassName('CIMSolidStroke', 'V3')
border_solidstroke.color = border_color
border_solidstroke.width = 3

background_solidfill     = arcpy.cim.CreateCIMObjectFromClassName('CIMSolidFill', 'V3')
background_solidfill.color.values = fill_color

# Create a polygon symbol 
polygon_symbol = arcpy.cim.CreateCIMObjectFromClassName('CIMPolygonSymbol', 'V3')
polygon_symbol.symbolLayers = [border_solidstroke, background_solidfill]

In [24]:
## Border ##

# Initialize border color 
border_color                = arcpy.cim.CreateCIMObjectFromClassName('CIMRGBColor', 'V3')
border_color.values         = brick_brown

# Create border symbol layer - set color and width  
border_solidstroke          = arcpy.cim.CreateCIMObjectFromClassName('CIMSolidStroke', 'V3')
border_solidstroke.color    = border_color
border_solidstroke.width    = 3

## Background ##

# Initialize fill color 
fill_color                  = arcpy.cim.CreateCIMObjectFromClassName('CIMRGBColor', 'V3')
fill_color.values           = trans_brown

# Create a fill symbol layer 
bg_solidfill                = arcpy.cim.CreateCIMObjectFromClassName('CIMSolidFill', 'V3')
bg_solidfill.color          = fill_color

# Create a polygon symbol 
polygon_symbol              = arcpy.cim.CreateCIMObjectFromClassName('CIMPolygonSymbol', 'V3')
polygon_symbol.symbolLayers = [border_solidstroke, bg_solidfill]

In [25]:
'''
Apply symbol to definition 
'''
lyr_name = 'dnr_stat_plan_areas_prk'

lyr            = mp.listLayers(lyr_name)[0]
lyr_definition = lyr.getDefinition('V3')

lyr_definition.renderer.symbol.symbol = polygon_symbol

lyr.setDefinition(lyr_definition)

In [305]:
aprx           = arcpy.mp.ArcGISProject("CURRENT")
lyr            = mp.listLayers(lyr_name)[0]
lyr_definition = lyr.getDefinition('V3')
lyr_symb_lyrs  = lyr_definition.renderer.symbol.symbol

print(lyr_symb_lyrs)

for d in dir(lyr_symb_lyrs): 
    if d.startswith("_") == False: 
        print(d)

angleAlignment
effects
symbolLayers
thumbnailURI
useRealWorldSymbolSizes
